# Imports and Definitions

In [1]:
pip install -q wandb

     |████████████████████████████████| 2.1MB 6.1MB/s 
     |████████████████████████████████| 163kB 18.8MB/s 
     |████████████████████████████████| 133kB 11.9MB/s 
     |████████████████████████████████| 102kB 10.6MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 


In [2]:
import numpy as np
import json
import glob

import tensorflow as tf
ks = tf.keras

import wandb
from wandb.keras import WandbCallback as WandbCallback

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Reading iNaturalist Data

In [4]:
path_train = "/content/drive/MyDrive/Sem 8/DL/inaturalist_12K/train"
path_test  = "/content/drive/MyDrive/Sem 8/DL/inaturalist_12K/val"

batch_size = 32
img_size = (800, 800)
img_shape = img_size + (3,)

train_ds = ks.preprocessing.image_dataset_from_directory(path_train,
                                                         label_mode='categorical',
                                                         validation_split = 0.1,
                                                         subset="training",
                                                         seed=123,
                                                         image_size=img_size,
                                                         batch_size=batch_size)

val_ds = ks.preprocessing.image_dataset_from_directory(path_train,
                                                       label_mode='categorical',
                                                       validation_split = 0.1,
                                                       subset="validation",
                                                       seed=123,
                                                       image_size=img_size,
                                                       batch_size=batch_size)

test_ds = ks.preprocessing.image_dataset_from_directory(path_test,
                                                        label_mode='categorical',
                                                        image_size=img_size,
                                                        batch_size=batch_size)

class_names = train_ds.class_names
num_classes = len(class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds   = val_ds.prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds  = test_ds.prefetch(buffer_size = tf.data.AUTOTUNE)

Found 10026 files belonging to 10 classes.
Using 9024 files for training.
Found 10026 files belonging to 10 classes.
Using 1002 files for validation.
Found 2000 files belonging to 10 classes.


In [5]:
AUTOTUNE

-1

# Functions

In [6]:
def finetune_model(hyp):
    
    # 1.
    model_to_use = hyp["model"]
    
    # 1.1. The function to get the model
    exec("ks_app_CNN_exec = ks.applications." + model_to_use[0])
    ks_app_CNN = locals()["ks_app_CNN_exec"]
    
    # 1.2. Supporting class for the model
    exec("ks_app_cnn_exec = ks.applications." + model_to_use[1])
    ks_app_cnn = locals()["ks_app_cnn_exec"]
    
    # 2.
    eta = hyp["eta"]
    epochs = hyp["epochs"]
    dropout = hyp["dropout"]
    num_unfrozen = hyp["num_unfrozen"]
    
    # 3.
    if hyp["include_top"]:
        # Creating the cnn
        cnn_model = ks_app_CNN(include_top=True,
                               weights='imagenet')
        img_width = cnn_model.layers[0].input_shape[0][1]
        
        # Defining data_augmentation function
        data_aug = ks.Sequential([
            ks.layers.experimental.preprocessing.RandomFlip('horizontal'),
            ks.layers.experimental.preprocessing.RandomRotation(0.2),
            ks.layers.experimental.preprocessing.Resizing(img_width, img_width, interpolation='bilinear')
        ])
    else:
        # Defining data_augmentation function
        data_aug = ks.Sequential([
            ks.layers.experimental.preprocessing.RandomFlip('horizontal'),
            ks.layers.experimental.preprocessing.RandomRotation(0.2)
        ])
        
        # Creating the cnn
        cnn_model = ks_app_CNN(input_shape=img_shape,
                               include_top=False,
                               weights='imagenet',
                               pooling='avg')
    
    # 4. Creating model with all layers in cnn frozen and fitting train data
    cnn_model.trainable = False

    # 4.1. Creating the model
    inputs = ks.Input(shape=img_shape)
    outputs = data_aug(inputs)
    outputs = ks_app_cnn.preprocess_input(outputs)
    outputs = cnn_model(outputs, training=False)
    outputs = tf.keras.layers.Dropout(dropout)(outputs)
    outputs = ks.layers.Dense(num_classes, activation='softmax')(outputs)

    model = ks.Model(inputs, outputs)
    model.compile(optimizer=ks.optimizers.Adam(lr=eta),
                  loss=ks.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    
    # 4.2. Fitting
    if num_unfrozen == 0:
        model.fit(train_ds,
                  epochs = epochs,
                  validation_data = val_ds, 
                  callbacks = [WandbCallback()])
    else:
        model.fit(train_ds,
                  epochs=epochs//2,
                  validation_data=val_ds,
                  callbacks = [WandbCallback()])
        
        # 5. Unfreezing num_unfrozen layers in cnn and fitting
        cnn_model.trainable = True
        for layer in cnn_model.layers[:-num_unfrozen]:
            layer.trainable = False

        model.compile(optimizer=ks.optimizers.Adam(lr=eta/10),
                      loss=ks.losses.categorical_crossentropy,
                      metrics=['accuracy'])
        
        model.fit(train_ds,
                  epochs=epochs,
                  validation_data=val_ds,
                  initial_epoch=epochs//2,
                  callbacks = [WandbCallback()])
        


def runSweep():
    wandb.init()
    
    #set the hyperparameters
    hyp = {}
    hyp["eta"] = wandb.config.eta
    hyp["epochs"] = wandb.config.epochs
    hyp["dropout"] = wandb.config.dropout
    hyp["model"] = wandb.config.model
    hyp["include_top"] = wandb.config.include_top
    hyp["num_unfrozen"] = wandb.config.num_unfrozen
    
    # Finetuning run
    finetune_model(hyp)

# Sweep

In [7]:
sweepCfg = {
    "name":"Fine Tuning Pretrained Sweep 7", 
    "method": "grid", 
    "parameters":{
        "include_top":{
            "values":[0, 1]
        },
        "num_unfrozen":{
            "values":[0, 5]
        },
        "model":{
            "values":[('Xception', 'xception'),
                    ('InceptionResNetV2', 'inception_resnet_v2'),
                    ('ResNet50', 'resnet50'),
                    ('InceptionV3', 'inception_v3')
                    ]
        },
        "eta":{
            "values":[1e-3]
        },
        "epochs":{
            "values":[10]
        },
        "dropout":{
            "values":[0.2]
        }
    }
}

sweepId = wandb.sweep(sweepCfg)
wandb.agent('srijan_gupta/uncategorized/u5j52d5c', function = runSweep)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 9luzz8lv
Sweep URL: https://wandb.ai/srijan_gupta/uncategorized/sweeps/9luzz8lv


wandb: Agent Starting Run: oadzvi39 with config:
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	eta: 0.001
wandb: 	include_top: 0
wandb: 	model: ['ResNet50', 'resnet50']
wandb: 	num_unfrozen: 5
wandb: Currently logged in as: srijan_gupta (use `wandb login --relogin` to force relogin)


94773248/94765736 [==============================] - 1s 0us/step
Epoch 1/5
282/282 [==============================] - 2985s 10s/step - loss: 1.6300 - accuracy: 0.4445 - val_loss: 0.9007 - val_accuracy: 0.7166
Epoch 2/5
282/282 [==============================] - 229s 797ms/step - loss: 0.9065 - accuracy: 0.7056 - val_loss: 0.7526 - val_accuracy: 0.7675
Epoch 3/5
282/282 [==============================] - 222s 774ms/step - loss: 0.7903 - accuracy: 0.7444 - val_loss: 0.6907 - val_accuracy: 0.7784
Epoch 4/5
282/282 [==============================] - 220s 765ms/step - loss: 0.7171 - accuracy: 0.7668 - val_loss: 0.6279 - val_accuracy: 0.8044
Epoch 5/5
282/282 [==============================] - 220s 766ms/step - loss: 0.6814 - accuracy: 0.7765 - val_loss: 0.6310 - val_accuracy: 0.8024
Epoch 6/10
282/282 [==============================] - 227s 779ms/step - loss: 0.6292 - accuracy: 0.7933 - val_loss: 0.5956 - val_accuracy: 0.8024
Epoch 7/10
282/282 [==============================] - 223s 776ms/

...

wandb: Agent Starting Run: 6xmht4kn with config:
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	eta: 0.001
wandb: 	include_top: 1
wandb: 	model: ['InceptionV3', 'inception_v3']
wandb: 	num_unfrozen: 5


Epoch 1/5
282/282 [==============================] - 193s 655ms/step - loss: 2.2704 - accuracy: 0.3493 - val_loss: 2.1738 - val_accuracy: 0.7455
Epoch 2/5
282/282 [==============================] - 187s 648ms/step - loss: 2.1553 - accuracy: 0.6846 - val_loss: 2.0579 - val_accuracy: 0.7695
Epoch 3/5
282/282 [==============================] - 185s 645ms/step - loss: 2.0507 - accuracy: 0.7095 - val_loss: 1.9487 - val_accuracy: 0.7745
Epoch 4/5
282/282 [==============================] - 186s 647ms/step - loss: 1.9498 - accuracy: 0.7061 - val_loss: 1.8466 - val_accuracy: 0.7774
Epoch 5/5
282/282 [==============================] - 187s 649ms/step - loss: 1.8586 - accuracy: 0.7033 - val_loss: 1.7508 - val_accuracy: 0.7745
Epoch 6/10
282/282 [==============================] - 192s 653ms/step - loss: 1.7124 - accuracy: 0.7104 - val_loss: 1.6000 - val_accuracy: 0.7964
Epoch 7/10
282/282 [==============================] - 187s 650ms/step - loss: 1.6247 - accuracy: 0.6967 - val_loss: 1.5602 - val_

epoch,9
loss,1.53126
accuracy,0.69581
val_loss,1.46782
val_accuracy,0.80339
_runtime,1891
_timestamp,1618411522
_step,9
best_val_loss,1.46782
best_epoch,9


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▇▆▅▄▃▂▁▁▁
accuracy,▁▇████████
val_loss,█▇▆▅▄▂▂▂▁▁
val_accuracy,▁▄▅▅▅▇▇███
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
